Install required packages

In [1]:
! pip install -qU google-cloud-aiplatform langchain chromadb pandas gradio

Restart the Runtime ! Runtime crashes so restart using the menu!

In [2]:
from google.cloud import aiplatform
aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project='ibm-keras',
    # the Vertex AI region you will use
    # defaults to us-central1
    location='us-central1',
)

Authenticate the user

In [3]:
import vertexai
import gradio as gr
import pandas as pd
from google.cloud import aiplatform
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.agents import create_pandas_dataframe_agent

PROJECT_ID = "ibm-keras"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

In [4]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Embedding
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [6]:
df = pd.read_csv('test.csv', sep=';')
print(df.head())

   age          job  marital  education default  balance housing loan  \
0   30   unemployed  married    primary      no     1787      no   no   
1   33     services  married  secondary      no     4789     yes  yes   
2   35   management   single   tertiary      no     1350     yes   no   
3   30   management  married   tertiary      no     1476     yes  yes   
4   59  blue-collar  married  secondary      no        0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome   y  
0  cellular   19   oct        79         1     -1         0  unknown  no  
1  cellular   11   may       220         1    339         4  failure  no  
2  cellular   16   apr       185         1    330         1  failure  no  
3   unknown    3   jun       199         4     -1         0  unknown  no  
4   unknown    5   may       226         1     -1         0  unknown  no  


In [7]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

In [8]:
def invoke_agent(query):
  return agent.run(query)

In [ ]:
def query_df(query):
    return invoke_agent(query)

customer_df_query_demo_app = gr.Interface(
    fn=query_df,
    inputs=gr.Textbox(lines=1, placeholder="Please Enter the customer search query here !"),
    outputs="text",
)
customer_df_query_demo_app.launch(share=True,debug=True)